In [2]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm


### States

In [93]:
zero  = np.array([[1], [0], [0], [0], [0], [0]],dtype=complex)
one   = np.array([[0], [1], [0], [0], [0], [0]],dtype=complex)
two   = np.array([[0], [0], [1], [0], [0], [0]],dtype=complex)
three = np.array([[0], [0], [0], [1], [0], [0]],dtype=complex)
four  = np.array([[0], [0], [0], [0], [1], [0]],dtype=complex)
five  = np.array([[0], [0], [0], [0], [0], [1]],dtype=complex)

### Permutations

In [94]:
SN = - zero@zero.T - one@one.T + five@two.T \
     +four@three.T+three@four.T+ two@five.T

SS =  -zero@zero.T - one@one.T + five@three.T \
     +four@two.T+two@four.T+ three@five.T

SE =  -two@two.T - three@three.T + five@one.T \
     +four@zero.T+zero@four.T+ one@five.T

SW = - two@two.T - three@three.T + five@zero.T \
     +four@one.T+one@four.T+ zero@five.T

D1 = zero@three.T + three@zero.T +one@two.T + two@one.T \
     -four@four.T - five@five.T

D2 = zero@two.T + two@zero.T +one@three.T + three@one.T \
     -four@four.T - five@five.T

rot_CW = zero@three.T + three@one.T + one@two.T + two@zero.T + four@five.T + five@four.T

rot_CCW = rot_CW.T

flip_V = zero@zero.T + one@one.T + three@two.T + two@three.T + four@five.T + five@four.T

flip_H = zero@one.T + one@zero.T + three@three.T + two@two.T + four@five.T + five@four.T

Id = zero@zero.T + one@one.T + two@two.T + \
     three@three.T + four@four.T + five@five.T

In [97]:
omega = np.exp(1j*2*np.pi/6)
#S_gate = np.matrix(np.diag([omega**0, omega**1, omega**3, omega**6,omega**10,omega**15]))
#states = [zero, one, two, three, four, five]
H = np.zeros((6,6),dtype=complex)
S_gate = np.zeros((6,6),dtype=complex)
for i in range(6):
    S_gate[i,i] = omega**(i*(i+1)/2)
    for j in range(6):
        H[i,j] = 1/math.sqrt(6) * omega**(j*i)


### Gate Set (S)

In [98]:
Perms = [SN,SS,SE,SN]#D1,D2,flip_H,flip_V,rot_CW,rot_CCW]
SQRT_SWAP = [(Id + 1j*P)/np.sqrt(2) for P in Perms]
Quarter_Swap = [np.cos(np.pi/8)*Id + np.sin(np.pi/8)*1j*P for P in Perms]
S = [S_gate,H]

#S.append(S_gate)

## Finding Adjoint representation in SO(35)

Generators of $\mathfrak{s u}(6)$ are found in mathematica via the approach found in https://mathematica.stackexchange.com/questions/159014/calculate-representations-of-sun-generators



In [99]:
G = []
# generate all the nondiagonal matricies
for i in range(6):
    for j in range(1,6-i):
        a = np.matrix(np.zeros((6,6),dtype=complex))
        b =  np.matrix(np.zeros((6,6)))
        a[i,i+j] = 1j/math.sqrt(2)
        a[i+j,i] = 1j/math.sqrt(2)
        b[i,i+j] = -1/math.sqrt(2)
        b[i+j,i] = 1/math.sqrt(2)
        G.append(a)
        G.append(b)
# generate the remaining diagonal generators
G.append(np.matrix(np.diag([1j/math.sqrt(2),-1j/math.sqrt(2),0,0,0,0])))
G.append(np.matrix(np.diag([1j/math.sqrt(6),1j/math.sqrt(6),-1j*math.sqrt(2/3),0,0,0])))
G.append(np.matrix(np.diag([1j/(2*math.sqrt(3)),1j/(2*math.sqrt(3)),1j/(2*math.sqrt(3)),-1j*math.sqrt(3)/2,0,0])))
G.append(np.matrix(np.diag([1j/(2*math.sqrt(5)),1j/(2*math.sqrt(5)),1j/(2*math.sqrt(5)),1j/(2*math.sqrt(5)),-2j/math.sqrt(5),0])))
G.append(np.matrix(np.diag([1j/(math.sqrt(30)),1j/(math.sqrt(30)),1j/(math.sqrt(30)),1j/(math.sqrt(30)),1j/(math.sqrt(30)),-1j*math.sqrt(5/6)])))

### Using the generators of the lie algebra to generate the adjoint representation of the matricies in our gate set

In [100]:
# generate the adjoint of an operator in SU(d) in SO(d^2-1)
def Ad(U,G):
    d = len(G)
    Ad_U = np.matrix(np.zeros((d,d),dtype=complex))
    for i in range(d):
        for j  in range(d):
            Ad_U[i,j] = -1/2 * np.trace(G[i]*U*G[j]*np.linalg.inv(U))
    return Ad_U

### Step 1 of the algorithm,

We need this code to print 1, which is the dimension of the kernal of the matrix $M_S$. If this is not onedimensional then it means that ther exists a set of matricies $A$ in the adjoint representation of $SU(6)$ that commute with every element of our gate set. This means that even though our set might be infinite we do not have a universal gate set. 

In [101]:
I = np.matrix(np.eye(len(G)))
Ms = np.matrix(np.zeros((len(G)**2*len(S),len(G)**2),dtype=complex))
for i,gate in enumerate(S):
    Ms[i*len(G)**2:(i+1)*len(G)**2,:] = np.kron(I,Ad(gate,G)) - np.kron(Ad(np.conj(gate.T),G),I)
# finding the kernal of MS
#Ms = np.matrix(Ms)
dim_ker = len(G)**2 - np.linalg.matrix_rank(Ms)
#ker = np.conj(Ms.T) @ Ms
print('The dimension of the kernal is', dim_ker)

The dimension of the kernal is 1


### Step 2 of the algorithm

In [102]:
def ball_check(gate,N,alphas):
    dim = gate.shape[0]
    for n in range(N):
        trace = np.trace(gate)
        for alpha in alphas:
            if (2*(dim) - alpha*np.conj(trace) - np.conj(alpha)*trace <= 1/math.sqrt(2)): # check if it is in B
                if (gate/gate[0,0] != np.eye(gate.shape[0])).all(): #check if it is in the center
                    return True
        gate = gate @ gate # increase the power by one
    return False

def test_close(A,B):
    dim = len(B[:,0])
    return np.allclose(np.abs(np.trace(np.conj(A.T)@B)),dim)

def add_unique(new_elems, group_elems):
    added = False
    for new_elem in new_elems:
        flag = False
        for group_elem in group_elems:
            if test_close(new_elem,group_elem):
                flag = True
                break
        if not(flag): 
            group_elems.append(new_elem)
            #group_elems.append(np.linalg.inv(new_elem))
            added = True
    return added

### Iterating steps 2 and 3

This code determines the span of our gate set is infinite or finite. It does this by attempting to find an element that can be reached from our gate set that is in a ball of radius 1 that is not in the center of the group. Not if this check pases, but the previous check fails that the span of $S$ is infinite, but not all of $SU(6)$, meaning it is not universal.  

In [103]:
# using a much smaller value for N that is required to be thourough since i want this to actually run 
N_SU6 = 100  #36398100 # upper bound for N
alphas = [np.exp(1j*2*np.pi/6*m) for m in range(6)]
G_s = [np.matrix(np.eye(6))]
flag = False
for l in range(10):# check words up to length 10 starting at l = 0
    new_gates = []
    for gate in G_s:
        if ball_check(gate,N_SU6,alphas):
            print('Infinite',gate) # if it is infinite also output the gate that is part of the ball and not the center
            flag = True
            break
        for U in S:
            new_gates.append(gate@U) # might be adding duplicate elements 
    if flag:
        break
    print('l = ',l)
    add_unique(new_gates,G_s)
            

l =  0
Infinite [[ 0.40824829+0.00000000e+00j  0.40824829+0.00000000e+00j
   0.40824829+0.00000000e+00j  0.40824829+0.00000000e+00j
   0.40824829+0.00000000e+00j  0.40824829+0.00000000e+00j]
 [ 0.40824829+0.00000000e+00j  0.20412415+3.53553391e-01j
  -0.20412415+3.53553391e-01j -0.40824829+1.58636328e-16j
  -0.20412415-3.53553391e-01j  0.20412415-3.53553391e-01j]
 [ 0.40824829+0.00000000e+00j -0.20412415+3.53553391e-01j
  -0.20412415-3.53553391e-01j  0.40824829-3.39934989e-16j
  -0.20412415+3.53553391e-01j -0.20412415-3.53553391e-01j]
 [ 0.40824829+0.00000000e+00j -0.40824829+1.58636328e-16j
   0.40824829-3.39934989e-16j -0.40824829+5.43895982e-16j
   0.40824829-7.25194643e-16j -0.40824829+8.83830971e-16j]
 [ 0.40824829+0.00000000e+00j -0.20412415-3.53553391e-01j
  -0.20412415+3.53553391e-01j  0.40824829-7.25194643e-16j
  -0.20412415-3.53553391e-01j -0.20412415+3.53553391e-01j]
 [ 0.40824829+0.00000000e+00j  0.20412415-3.53553391e-01j
  -0.20412415-3.53553391e-01j -0.40824829+8.8383097

C:\Users\Noah\AppData\Local\Temp\ipykernel_19564\1977419837.py:9: RuntimeWarning: overflow encountered in matmul
  gate = gate @ gate # increase the power by one
C:\Users\Noah\AppData\Local\Temp\ipykernel_19564\1977419837.py:9: RuntimeWarning: invalid value encountered in matmul
  gate = gate @ gate # increase the power by one
C:\Users\Noah\AppData\Local\Temp\ipykernel_19564\1977419837.py:6: RuntimeWarning: invalid value encountered in cdouble_scalars
  if (2*(dim) - alpha*np.conj(trace) - np.conj(alpha)*trace <= 1/math.sqrt(2)): # check if it is in B


# Qubit Version of the Code

In [72]:
zero  = np.array([[1], [0]],dtype=complex)
one   = np.array([[0], [1]],dtype=complex)

In [88]:
S_gate = np.matrix(np.diag([1,1j]))
H = 1/math.sqrt(2)*np.matrix([[1,1],[-1,1]])
S = [S_gate,H]

In [89]:
X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,-1j],[1j,0]])
Z = np.matrix(np.diag([1,-1]))
G = [X,Y,Z]

In [90]:
I = np.matrix(np.eye(len(G)))
Ms = np.matrix(np.zeros((len(G)**2*len(S),len(G)**2),dtype=complex))
for i,gate in enumerate(S):
    Ms[i*len(G)**2:(i+1)*len(G)**2,:] = np.kron(I,Ad(gate,G)) - np.kron(Ad(np.conj(gate.T),G),I)
# finding the kernal of MS
#Ms = np.matrix(Ms)
dim_ker = len(G)**2 - np.linalg.matrix_rank(Ms)
#ker = np.conj(Ms.T) @ Ms
print('The dimension of the kernal is', dim_ker)

The dimension of the kernal is 1


In [92]:
# using a much smaller value for N that is required to be thourough since i want this to actually run 
N_SU2 = 6  #36398100 # upper bound for N
alphas = [np.exp(1j*2*np.pi/2*m) for m in range(6)]
G_s = [np.matrix(np.eye(2))]
flag = False
for l in range(10):# check words up to length 10 starting at l = 0
    new_gates = []
    for gate in G_s:
        if ball_check(gate,N_SU2,alphas):
            print('Infinite',gate) # if it is infinite also output the gate that is part of the ball and not the center
            flag = True
            break
        for U in S:
            new_gates.append(gate@U) # might be adding duplicate elements 
    if flag:
        break
    print('l = ',l)
    if not(add_unique(new_gates,G_s)):
        break
print(len(G_s))


l =  0
l =  1
l =  2
l =  3
l =  4
l =  5
24
